# Лабораторна робота №7 ІАД

<b>Виконала:</b> студентка МІТ-31, Панченко Владислава

<b>Завдання:</b>
1. Виконати вирішення задач регресії та класифікації для наборів даних, що використовувалися в лабораторних роботах 3 та 5 з використанням Tensorflow & NN.
2. Отримані результати викласти на github у репозиторій в основну (default) гілку в папці Lab7.


Перший набір даних:

In [29]:
import tensorflow as tf
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from setuptools.dist import Distribution

data = pd.read_csv('gendergapinaverage.csv', encoding='windows-1252')
df = pd.DataFrame(data)

X = df[['Year']]
y = df['Gender wage gap %']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_regression = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

model_regression.compile(optimizer='adam', loss='mean_squared_error')
model_regression.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test), verbose=0)

y_pred_nn = model_regression.predict(X_test_scaled)

mse_nn = mean_squared_error(y_test, y_pred_nn)
mae_nn = mean_absolute_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

print(f"Середньоквадратична похибка (MSE): {mse_nn}")
print(f"Середня абсолютна похибка (MAE): {mae_nn}")
print(f"Коефіцієнт детермінації (R^2): {r2_nn}")

threshold = 20
y_class = (y > threshold).astype(int)
X_train, X_test, y_train_class, y_test_class = train_test_split(X, y_class, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_classification = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') 
])
model_classification.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_classification.fit(X_train_scaled, y_train_class, epochs=100, batch_size=32, validation_split=0.2, verbose=0)

y_pred_class = model_classification.predict(X_test_scaled)
y_pred_class = (y_pred_class > 0.5).astype(int)

accuracy = (y_pred_class.flatten() == y_test_class.values).mean()
print(f"Точність класифікаційної моделі: {accuracy}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Середньоквадратична похибка (MSE): 98.066844077369
Середня абсолютна похибка (MAE): 7.94360416504274
Коефіцієнт детермінації (R^2): 0.06816176970621557
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Точність класифікаційної моделі: 0.8313253012048193


Другий набір даних:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from setuptools.dist import Distribution

data = pd.read_csv("16P.csv", encoding='windows-1252')
data = data.sample(frac=0.1, random_state=42)

X = data.drop(columns=["Response Id", "Personality"]) 
y = data["Personality"]

for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])
if y.dtype == 'object':
    y = LabelEncoder().fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model_regression = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),  
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

model_regression.compile(optimizer='adam', loss='mean_squared_error')
model_regression.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=0)
y_pred_nn = model_regression.predict(X_test)
mse_nn = mean_squared_error(y_test, y_pred_nn)
mae_nn = mean_absolute_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

print(f"Середньоквадратична похибка (MSE): {mse_nn}")
print(f"Середня абсолютна похибка (MAE): {mae_nn}")
print(f"Коефіцієнт детермінації (R^2): {r2_nn}")

threshold = 20
y_class = (y > threshold).astype(int)
X_train, X_test, y_train_class, y_test_class = train_test_split(X_scaled, y_class, test_size=0.2, random_state=42)

model_classification = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)), 
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') 
])

model_classification.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_classification.fit(X_train, y_train_class, epochs=100, batch_size=32, validation_split=0.2, 
                         callbacks=[early_stopping], verbose=0)

y_pred_class = model_classification.predict(X_test)  
y_pred_class = (y_pred_class > 0.5).astype(int) 
accuracy = (y_pred_class.flatten() == y_test_class.flatten()).mean()
print(f"Точність: {accuracy}")

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Середньоквадратична похибка (MSE): 5.950985432883844
Середня абсолютна похибка (MAE): 1.893760682096084
Коефіцієнт детермінації (R^2): 0.7220732569694519
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Точність: 1.0
